In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing necessary packages**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image,ImageOps
from tqdm import tqdm
from sklearn.utils import shuffle

# **Exploring the dataset**

In [3]:
train_dir="/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/"
print("Length of training directory {}".format(len(os.listdir(train_dir))))

In [4]:
test_dir="/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/"
print("Length of training directory {}".format(len(os.listdir(test_dir))))

# **Loading the Training dataset**

In [5]:
tot_images=[]
tot_labels=[]

In [6]:
def label(s):
    if len(s)==1:
        return ord(s)-65
    elif s=="nothing":
        return 26
    elif s=="space":
        return 27
    else:
        return 28

In [7]:
for i in tqdm(os.listdir(train_dir)):
    for image in os.listdir(os.path.join(train_dir,i)):
        img=Image.open(os.path.join(os.path.join(train_dir,i),image))
        img=ImageOps.grayscale(img)
        img=img.resize((64,64))
        img=np.asarray(img)
        img=img.reshape((64,64,1))
        tot_images.append(img)
        k=label(i)
        tot_labels.append(k)

In [8]:
print(len(tot_images),len(tot_labels))

# **Converting lists to arrays**

In [9]:
tot_images=np.array(tot_images)

In [10]:
tot_labels=np.array(tot_labels)

# **Normalisation of dataset**

In [11]:
tot_images=tot_images/255

In [12]:
tot_images[0]

# **Shuffling the dataset**

In [13]:
tot_images,tot_labels=shuffle(tot_images,tot_labels)

In [15]:
tot_images[0],tot_labels[0]

# **Splitting the dataset into train and test sets**

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(tot_images,tot_labels,test_size=0.3,stratify=tot_labels)

In [18]:
print(len(x_train),len(x_test),len(y_train),len(y_test))

In [19]:
print(np.unique(y_test,return_counts=True))

# **Building the model**

In [20]:
import keras

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical

In [22]:
y_train_conv = to_categorical(y_train, 29)
y_test_conv = to_categorical(y_test, 29)

In [34]:
input_tensor = Input(shape=(64,64,1))
vgg=VGG16(weights="imagenet",include_top=False)
x = Conv2D(3,(3,3),padding='same')(input_tensor)
vgg=vgg(x)
base_model = Model(inputs=input_tensor,outputs=vgg)

In [35]:
base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [36]:
model=Sequential()
model.add(base_model)
output=base_model.layers[-1].output
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(29, activation='softmax'))
o=Adam(learning_rate=0.001)

In [37]:
o=Adam(learning_rate=0.001)

In [38]:
model.compile(optimizer=o,loss="categorical_crossentropy",metrics=['accuracy'])

In [39]:
model.fit(x_train,y_train_conv,epochs=20,verbose=1)

In [40]:
predict_x=model.predict(x_test) 
classes_x=np.argmax(predict_x,axis=1)

In [41]:
y_hat=model.predict_classes(x_test)

In [42]:
classes_x[0]

In [43]:
from sklearn.metrics import accuracy_score,f1_score
print("Accuracy of the model : ",accuracy_score(y_test,classes_x))

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
from sklearn.metrics import precision_score,recall_score

In [46]:
print(precision_score(y_test,classes_x,pos_label='positive',average='micro'))

In [47]:
print(recall_score(y_test,classes_x,pos_label='positive',average='micro'))

In [48]:
print("F1 score of the model : ",f1_score(y_test,classes_x,pos_label='positive',average='micro'))

In [ ]:
cf=confusion_matrix(y_test,classes_x)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(cf,annot=True)

# **Test dataset**

In [ ]:
test_list=[]
test_labels=[]

In [ ]:
for image in tqdm(os.listdir(test_dir)):
    img=Image.open(os.path.join(test_dir,image))
    img=ImageOps.grayscale(img)
    img=img.resize((64,64))
    img=np.asarray(img)
    img=img.reshape((64,64,1))
    test_list.append(img)
    test_labels.append(label(image.split('_')[0]))

In [ ]:
print(test_list[0],test_labels[0])

In [ ]:
test_list=np.array(test_list)
test_list=test_list/255

In [ ]:
predict_x=model.predict(test_list) 
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
accuracy_score(test_labels,classes_x)

# **Downloading the model**

In [ ]:
model.save('/kaggle/working/model.h5')

In [49]:
model.save('/kaggle/working/model_color.h5')